https://www.verizon.com/about/news/news-releases

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import sys,bs4,re,time,requests,pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install()
#                           ,options=option
                         )



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [3]:
def parse_webpage_bs(search_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [4]:
url = f'https://www.verizon.com/about/news/news-releases'

driver.maximize_window()

driver.get(url)

hrefs = []

range_count = 3

for x in range(range_count):
    driver.implicitly_wait(100)
    driver.find_element_by_class_name('''nav-right_arrow ''').click()                                  
    for y in range(1,11):
        link = driver.find_element_by_xpath(f'''/html/body/div[2]/main/div/div/div[2]/div[2]/div/div/div/div/div/div[2]/div[2]/div[{y}]/div[3]/h2/a''').get_attribute("href")
        hrefs.append(link)                               
    print(str(range_count-(x+1)) + " pages remaining..",end="\r")

In [5]:
articles = []
dates = []
titles = []
thumbnails = []

for number,link in enumerate(hrefs):
    
    soup = parse_webpage_bs(link)

    t_containers = soup.findAll('div',{'class' :'news-fp-heading'})
    
    for t in t_containers:
        title = t.text
        titles.append(title)
    
    d_containers = soup.findAll('div',{'class' :'news-fp-date'})
    
    for d in d_containers:
        date = d.find('span').text
        dates.append(date)
        
    thumb_containers = soup.findAll('div',{'class' :'news-fp-image'})
    
    if len(thumb_containers) != 0:
        for thumb in thumb_containers:
            thumbnail = "https://www.verizon.com" + thumb.find('img')['src']
            thumbnails.append(thumbnail)
    else:
        thumbnails.append("")

    art_containers = soup.findAll('div',{'class' :'card'})
    
    for art in art_containers:
        article = art.text
        articles.append(article)
        
    print(str(len(hrefs)-(number+1)) + " url remaining..",end="\r")

    
print()

0 url remaining...


In [6]:
#zippling the list so that it'll form the tuple
zipped = list(zip(dates,titles,articles,hrefs,thumbnails))
zipped[:3]

[('04.25.2022',
  'Verizon makes BOPIS bigger with BOPIL ',
  "04.25.2022|Products & Plans|Home & EntertainmentVerizon makes BOPIS bigger with BOPIL  Media contact(s)Heidi Flatoheidi.flato@verizon.com Express Pickup Lockers rolling out to stores nationwide to meet increased customer demand for convenienceFull Transparency No UpdatesLearn moreWe're committed to building trust.The Verizon Newsroom greatly values transparency. By integrating blockchain technology, we’re able to permanently log all changes made to official releases after publication. We believe people deserve the highest level of integrity. And we’re committed to setting the industry standard for corporate communications.Learn moreBASKING RIDGE, NJ - Today’s consumers want convenience and Verizon continues to look at new and innovative ways to meet them when, where and how they want to be met. After the past two years, people have become familiar with a hybrid retail model known as buy online pickup in store (BOPIS) but no

In [7]:
#creating dataframe 
temp_df = pd.DataFrame(zipped,columns=['date','title','article','url','thumbnail'])
temp_df.head()

,date,title,article,url,thumbnail
0,04.25.2022,Verizon makes BOPIS bigger with BOPIL,04.25.2022|Products & Plans|Home & Entertainme...,https://www.verizon.com/about/news/verizon-mak...,
1,04.22.2022,Verizon reports 1Q earnings for 2022,"BASKING RIDGE, NJ - Today’s consumers want con...",https://www.verizon.com/about/news/verizon-rep...,https://www.verizon.com/about/sites/default/fi...
2,04.21.2022,"Verizon appoints Christina Schelling leader, T...",04.22.2022|Financial|EarningsVerizon reports 1...,https://www.verizon.com/about/news/verizon-app...,https://www.verizon.com/about/sites/default/fi...
3,04.21.2022,"Verizon to report earnings April 22, 2022",Download News Release PDFDownload Infographic ...,https://www.verizon.com/about/news/verizon-rep...,
4,04.20.2022,HBO Max joins Verizon’s +play,04.21.2022|Financial|CorporateVerizon appoints...,https://www.verizon.com/about/news/hbo-max-joi...,


In [8]:
#to csv
temp_df.to_csv('salesforce news.csv')

#to json
temp_df.to_json('salesforce news.json')